# AIM

ニューラルネットワークで排他的論理和回路のモデル ver. softmax with TensorFlow

In [1]:
import datetime
import numpy as np
import tensorflow as tf

In [2]:
from pkg_resources import get_distribution
import platform
print("python", platform.python_version())
print("")
libs = ["numpy", "tensorflow"]
for lib in libs:
    version = get_distribution(lib).version
    print(lib, version)

python 3.5.2

numpy 1.13.1
tensorflow 1.3.0


In [4]:
# モデルクラス定義
class NN():
    def __init__(self, in_size, hidden_size, out_size):
        # クラスの初期化
        # :param in_size: 入力層のサイズ
        # :param hidden_size: 隠れ層のサイズ
        # :param out_size: 出力層のサイズ
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        # プレースホルダー
        self.placeholder_x = tf.placeholder(tf.float32, shape=[None, self.in_size], name="placeholder_x")
        self.placeholder_t = tf.placeholder(tf.float32, shape=[None, self.out_size], name="placeholder_t")
        # 順伝播のネットワークを作成する
        self.b1 = tf.Variable(tf.random_normal([self.hidden_size]), name="b1") # 入力層のバイアス
        self.w1= tf.Variable(tf.random_normal([self.in_size, self.hidden_size]), name="w1") # 入力層の重み
        self.h1 = tf.sigmoid(tf.matmul(self.placeholder_x, self.w1) + self.b1)
        self.b2 = tf.Variable(tf.random_normal([self.hidden_size]), name="b2") # 隠れ層のバイアス
        self.w2 = tf.Variable(tf.random_normal([self.hidden_size, self.hidden_size]), name="w2") # 隠れ層の重み
        self.h2 = tf.sigmoid(tf.matmul(self.h1, self.w2) + self.b2)
        self.b3 = tf.Variable(tf.random_normal([self.out_size]), name="b2") # 出力層のバイアス
        self.w3 = tf.Variable(tf.random_normal([self.hidden_size, self.out_size]), name="w3") # 出力層の重み
        self.y = tf.nn.softmax(tf.matmul(self.h2, self.w3) + self.b3)
        # クロスエントロピーを計算する
        self.cross_entropy = -tf.reduce_sum(self.placeholder_t*tf.log(self.y))
        # 勾配を計算する
        self.optimizer = tf.train.AdamOptimizer(0.01)
        self.train_op = self.optimizer.minimize(self.cross_entropy)
        # 正確度を計算する
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.y, 1), tf.argmax(self.placeholder_t, 1)), "float"))
        # セッション開始
        self.init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(self.init) 
    
    def __call__(self, x, t=None, train=False):
        # 順伝播の計算を行う関数
        # :param x: 入力値
        # :param t: 正解のラベル
        # :param train: 学習かどうか
        # :return: 計算した損失 or 予測したラベル
        fetches = self.y
        feed_dict = {}
        feed_dict[self.placeholder_x] = x
        if train:
            fetches = [self.train_op, self.cross_entropy, self.accuracy, self.b1, self.w1, self.b2, self.w2, self.b3, self.w3]
            feed_dict[self.placeholder_t] = t
        cvalues = self.sess.run(fetches, feed_dict=feed_dict) # 計算
        if train:
            _, loss, accuracy = cvalues[0], cvalues[1], cvalues[2] # 損失と正確度だけ返却する
            return loss, accuracy
        else:
            return np.argmax(cvalues) # cvaluesにself.yの計算結果が入る


In [12]:
# 学習
EPOCH_NUM = 2000
HIDDEN_SIZE = 5
BATCH_SIZE = 4

# 教師データ
train_data = [
    [[0, 0], [0]],
    [[1, 0], [1]],
    [[0, 1], [1]],
    [[1, 1], [0]]
]

# 教師データを変換
in_size = len(train_data[0][0]) #  入力サイズ
out_size = in_size # 出力サイズ
N = len(train_data) # 教師データの総数
train_x, train_t = [], []
for x, t in train_data:
    train_x.append(x)
    t_ = np.zeros(out_size)
    t_[t[0]] = 1
    train_t.append(t_)
train_x = np.array(train_x, dtype="float32")
train_t = np.array(train_t, dtype="float32")

# モデルの定義
model = NN(in_size=in_size, hidden_size=HIDDEN_SIZE, out_size=out_size)

# 学習開始
print("Train")
st = datetime.datetime.now()
for epoch in range(EPOCH_NUM):
    # ミニバッチ学習
    perm = np.random.permutation(N) # ランダムな整数列リストを取得
    total_loss = 0
    total_accuracy = 0
    for i in range(0, N, BATCH_SIZE): 
        x = train_x[perm[i:i+BATCH_SIZE]]
        t = train_t[perm[i:i+BATCH_SIZE]]
        loss, accuracy = model(x=x, t=t, train=True)
        total_loss += loss
        total_accuracy += accuracy
    if (epoch+1) % 500 == 0:
        ed = datetime.datetime.now()
        print("epoch:\t{}\ttotal loss:\t{}\tmean accuracy:\t{}\ttime:\t{}".format(epoch+1, total_loss, total_accuracy/(N/BATCH_SIZE), ed-st))
        st = datetime.datetime.now()

Train
epoch:	500	total loss:	0.03256687521934509	mean accuracy:	1.0	time:	0:00:00.631843
epoch:	1000	total loss:	0.008058303035795689	mean accuracy:	1.0	time:	0:00:00.611837
epoch:	1500	total loss:	0.0038142483681440353	mean accuracy:	1.0	time:	0:00:00.624347
epoch:	2000	total loss:	0.0022085041273385286	mean accuracy:	1.0	time:	0:00:00.616167


In [13]:
# 予測
print("\nPredict")
def predict(model, x):
    y = model(x=[x], train=False)
    print("x:\t{}\t=>\ty:\t{}".format(x, y))

# 予測
predict(model, [1, 0])
predict(model, [0, 0])
predict(model, [1, 1])
predict(model, [0, 1])


Predict
x:	[1, 0]	=>	y:	1
x:	[0, 0]	=>	y:	0
x:	[1, 1]	=>	y:	0
x:	[0, 1]	=>	y:	1
